In [3]:
# 本文内容翻译从
# Manav Sehgal的titanic-data-science-solutions
# 有兴趣的可以去url看看英文原版 https://www.kaggle.com/startupsci/titanic-data-science-solutions
# 以下就是我们开始入门kaggle的titanic实战代码啦
# 由于本人水平有限，有错的地方欢迎看到的人指出
# 我尽量以一种很通俗易懂的方式让我们明白每一步的意义

In [4]:
# 首先第一步挂在数据进你的kernel，加载完成后，系统会提示你数据的位置
import pandas as pd
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")

In [5]:
# kernel主要的七个步骤内容
# 1.分析问题（预测的问题）
# 2.分析训练数据和测试数据（https://www.kaggle.com/c/titanic/data）
#     着重看我们的变量有哪些，比如关于sex性别这一属性对吧，是男是女很有影响的
#     一般发生这样的事情都是女士优先的，还有age也是很重要的权重属性
# 3.数据预处理
# 4.分析，识别模式，探索应用数据
# 5.开始建模，预测，解决问题（这里是核心的核心）
# 6.可视化，完成报告，提出问题的解决步骤和最后结果
# 7.提交结果，进行竞赛排行(LB)





In [6]:
#数据分析用到的库
import pandas as pd
import numpy as np
import random as rnd
# 数据可视化用到的库
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#机器学习用到的sklearn库
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



In [7]:
#载入数据进去(数据的路径在开头处给与)
train_df=pd.read_csv("../input/titanic/train.csv")
test_df=pd.read_csv("../input/titanic/test.csv")
combine=[train_df,test_df]
#combine函数将两个数据相结合 

In [8]:
#载入进数据之后就要分析数据

#打印特征变量标签
print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [9]:
#首先分析那些特征(feature)是明确的(categorical)
#所谓明确的就是指，不是1就是2的那种特征属性，比如我们的性别，不是男的就是女的
#比如我们的票仓，并不是一等舱就是二等舱，三等舱
#Categorical: Survived, Sex, and Embarked. Ordinal: Pclass.

#再分析哪些特征(feature)是量化的(numerical)
#所谓量化(emm自己编的)就是指，它的取值可以是很多不等，没有具体值的一些属性
#比如我们的年龄，我们的恐惧值，它都是0.00到100.00浮点不值的数值
#Continous: Age, Fare. Discrete: SibSp, Parch.


#预览一部分数据
train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
#通过预览图我们可以发现有的数据浮点数很高，比如FARE值，这显然是不利于我们做数据模型的
#通过预览图，我们以可以发现有的诗句是存在错误的

In [11]:
#我们再来预览尾部数据
#通过尾部预览图我们能从中得出训练数据一共有890个样本
train_df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [12]:
#打印出数据整体信息
#通过整体信息图我们还可以得出数据有缺失的情况存在NaN(NOt A NUMBER)--这些情况我们必须纠正
#试想不纠正的话，做平均值处理的时候会得到一个很错误的结果，毕竟890个样本不变，而你却没有对应那个样本的数据进行求和

train_df.info()
print('_'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null

In [13]:
#从整体表中，我们可以知道Cabin(机舱号)这一特征缺失的特别厉害
#年龄也缺失得很严重

#还需要观察dtype属性(毕竟做数据运算的时候，你不可能用object属性去做加法运算)


In [14]:
# 接下来我们研究数据的分布情况
# 这点超级重要，能够体现数据的集中属性，帮助我们做一下直观的判断
# 哪些属性对结果也就是(Survive)影响较大
train_df.describe()
# 样本总数为891人，占泰坦尼克号（2,224）旅客实际人数的40％。

# 生存是具有0或1个值的分类特征。

# 约有38％的样本存活下来，代表了32％的实际存活率。

# 大多数乘客（> 75％）没有和父母或孩子一起旅行。

# 将近30％的乘客有兄弟姐妹和/或配偶。

# 票价差异很大，只有极少数乘客（<1％）支付的费用高达512美元。

# 65-80岁年龄段的老年乘客很少（<1％）。

#Ps 这里编者着重说明 统计学符号的含义
# count ：计数的意思
# mean ：求均值(有的属性均值毫无意义，比如乘客ID，担忧的却十分重要比如fare值，存活值，这里可以看出存活率不足0.4)
# std ：标准偏差--度量数据分布的分散程度之标准(代商榷，因为我也不是很了解哈哈哈)
# min :最小值
# 下面都是4分位的那个乘客的特征情况
# (这个有利于做统计学的运算)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [15]:
train_df.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Ilmakangas, Miss. Pieta Sofia",male,CA. 2343,B96 B98,S
freq,1,577,7,4,644


In [20]:
# 接下来就是数据预处理
# 这是一个相当重要的环节，如果数据预处理得好，通过固定的模型算法，得到的结果将会是相当可观的
# 数据预处理的步骤

# 相关性。
# 我们想知道每个功能与生存率的关联程度。 我们希望在项目的早期进行此操作，并将这些快速相关性与项目后期的建模相关性进行匹配。

# 完整性
# 我们可能希望完成“年龄”功能，因为它肯定与生存相关。
# 我们可能要完成“登载”功能，因为它也可能与生存或其他重要功能相关。

# 纠正
# 票证功能可能会从我们的分析中删除，因为它包含很高的重复率（22％），并且票证与生存率之间可能没有关联。
# 客舱功能可能由于高度不完整或在训练和测试数据集中包含许多空值而被删除。
# 旅客ID可能不会从训练数据集中删除，因为它对生存没有帮助。
# 名称功能是相对非标准的，可能不会直接有助于生存，因此可能会被放弃。

# 创造
# 我们可能希望基于Parch和SibSp创建一个称为“家庭”的新功能，以获取船上家庭成员的总数。
# 我们可能要设计名称功能，以将标题提取为新功能。
# 我们可能要为年龄段创建新功能。 这会将连续的数字特征转换为有序的分类特征。
# 如果它有助于我们的分析，我们可能还想创建一个票价范围功能。

# 判别性(一种基于问题明锐思想)
# 我们也可以根据前面提到的问题描述增加假设。
# 女性（性=女性）更有可能存活下来。
# 儿童（年龄<？）存活的可能性更高。
# 上等乘客（Pclass = 1）更有可能幸存下来。

# 相关性分析的时候，我们通过旋转特征进行分析
# 为了确认我们的一些观察和假设，我们可以通过使要素相互对立来快速分析要素相关性。 我们现阶段只能对没有任何空值的要素执行此操作。 仅对分类（Sex），序数（Pclass）或离散（SibSp，Parch）类型的特征这样做也是有意义的。
# Pclass我们观察到Pclass = 1和Survived（分类＃3）之间的显着相关性（> 0.5）。 我们决定在模型中包括此功能。
# 性别我们确认在问题定义期间的观察结果是，性别=女性具有很高的生存率，为74％（分类1）。
# SibSp和Parch这些功能对于某些值具有零相关性。 最好是从这些单个要素中派生一个要素或一组要素（创建＃1）。

train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# 可以看到Pclass(票价等级)对于船员生存来说具有极大相关性


,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [21]:
train_df[['Sex','Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived',ascending=False) 
#同理


,Sex,Survived
0,female,0.742038
1,male,0.188908


In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)